In [1]:
# Import libraries
import os
import json
import pandas as pd
import numpy as np
import nltk
import gensim
import re
import torch
import torchvision
from sklearn.model_selection import train_test_split
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from gensim.models import Word2Vec
from transformers import AutoTokenizer, AutoModelForSequenceClassification, AutoConfig
from datasets import load_dataset, DatasetDict, load_from_disk
from collections import Counter

# # Download NLTK resources
# nltk.download('punkt_tab')
# nltk.download('stopwords')
# nltk.download('wordnet')

In [2]:
# Load the LIAR dataset
# dataset = "chengxuphd/liar2"
# dataset = load_dataset(dataset)
dataset = load_from_disk("./oversampled_datasets")
# train_dataset = load_dataset("csv", data_files="../data/train.csv")["train"]
# val_dataset = load_dataset("csv", data_files="../data/val.csv")["train"]
# test_dataset = load_dataset("csv", data_files="../data/test.csv")["train"]
# dataset = DatasetDict({
#     "train": train_dataset,
#     "validation": val_dataset,
#     "test": test_dataset
# })

pretrained_model = "roberta-base"
tokenizer = AutoTokenizer.from_pretrained(pretrained_model)

# Combine metadata with the preprocessed statement
def preprocess_function(examples):
    combined_input = [
        "Subject: " + (subject if subject is not None else "") + 
        "; Speaker: " + (speaker if speaker is not None else "") + 
        "; Speaker Description: " + (speaker_description if speaker_description is not None else "") + 
        "; State: " + (state_info if state_info is not None else "") + 
        "; Context: " + (context if context is not None else "") + 
        "; Statement: " + (statement if statement is not None else "") 
        for subject, speaker, speaker_description, state_info, context, statement in zip(
            examples["subject"],
            examples["speaker"],
            examples["speaker_description"],
            examples["state_info"],
            examples["context"],
            examples["statement"]
        )
    ]
    return tokenizer(combined_input, padding="max_length", truncation=True)

# Apply tokenization to the dataset
tokenized_datasets = dataset.map(preprocess_function, batched=True)

# Remove unnecessary columns and set format for PyTorch
tokenized_datasets = tokenized_datasets.remove_columns(["id", "subject", "speaker", "speaker_description", "state_info", "context", "true_counts", "mostly_true_counts", "half_true_counts", "mostly_false_counts", "false_counts", "pants_on_fire_counts", "justification"])
# tokenized_datasets = tokenized_datasets.rename_column("label", "labels")

# label_to_binary = {
#     # label_to_id = {"REAL": 0, "FAKE": 1}
#     0: 1,
#     1: 1,
#     2: 1,
#     3: 1,  # changed
#     4: 0,
#     5: 0
# }

# original_label_counts = Counter(dataset["train"]["label"])
# print("Before label preprocessing: Labels =", original_label_counts)

# # Apply binary label preprocessing
# tokenized_datasets = tokenized_datasets.map(
#     lambda examples: {"label": [label_to_binary[int(label)] for label in examples["label"]]},
#     batched=True
# )

tokenized_datasets.set_format("torch")

# Split into train, validation, and test sets
train_dataset = tokenized_datasets["train"]
val_dataset = tokenized_datasets["validation"]
test_dataset = tokenized_datasets["test"]

# binary_label_counts = Counter(train_dataset["label"].tolist())
# print("After label preprocessing: Labels =", binary_label_counts)

Map:   0%|          | 0/1204 [00:00<?, ? examples/s]

In [3]:
# Verify that dataset is preprocessed correctly

# Check label distribution in the train dataset
label_list = train_dataset["label"].tolist()
print("Label distribution in train dataset:", Counter(label_list))

Label distribution in train dataset: Counter({1: 13621, 0: 13495})


In [4]:
from transformers import TrainingArguments, Trainer, EarlyStoppingCallback
from sklearn.metrics import accuracy_score, precision_recall_fscore_support

# Define compute_metrics function
def compute_metrics(pred):
    labels = pred.label_ids
    preds = pred.predictions.argmax(-1)
    precision, recall, f1, _ = precision_recall_fscore_support(labels, preds, average="binary")
    acc = accuracy_score(labels, preds)
    return {
        "accuracy": acc,
        "f1": f1,
        "precision": precision,
        "recall": recall,
    }

# Define training arguments (epoch)
training_args = TrainingArguments(
    output_dir="./results",
    evaluation_strategy="epoch",
    save_strategy="epoch",
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=5,
    weight_decay=0.01,
    learning_rate=2e-5,
    lr_scheduler_type="cosine",
    warmup_steps=500,
    logging_dir="./logs",
    logging_steps=10,
    report_to="none",
    load_best_model_at_end=True,  # Load the best model at the end of training
    metric_for_best_model="recall",  # Specify the metric to monitor
    greater_is_better=True       # Specify if higher values of the metric are better
)

# # Define training arguments (steps for smaller batch logging)
# training_args = TrainingArguments(
#     output_dir="./results",
#     evaluation_strategy="steps",  # Evaluate during training
#     eval_steps=100,               # Evaluate every 100 steps
#     save_strategy="steps",        # Save checkpoints every 100 steps
#     save_steps=100,
#     per_device_train_batch_size=16,
#     per_device_eval_batch_size=16,
#     num_train_epochs=3,
#     weight_decay=0.01,
#     learning_rate=2e-5,
#     logging_dir="./logs",
#     logging_steps=10,             # Log every 10 steps
#     report_to="none",
#     load_best_model_at_end=True,  # Load the best model at the end of training
#     metric_for_best_model="f1",  # Specify the metric to monitor
#     greater_is_better=True       # Specify if higher values of the metric are better
# )

c:\Users\limyi\AppData\Local\Programs\Python\Python311\Lib\site-packages\transformers\training_args.py:1611: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [5]:
config = AutoConfig.from_pretrained(pretrained_model, num_labels=2, hidden_dropout_prob=0.2, attention_probs_dropout_prob=0.2)
model = AutoModelForSequenceClassification.from_pretrained(pretrained_model, config=config)
tokenizer = AutoTokenizer.from_pretrained(pretrained_model)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)

# Swap the labels in the model configuration
model.config.id2label = {0: "REAL", 1: "FAKE"}
model.config.label2id = {"REAL": 0, "FAKE": 1}

# Verify the changes
print("Updated id2label mapping:", model.config.id2label)
print("Updated label2id mapping:", model.config.label2id)

training_args.num_train_epochs = 5
training_args.learning_rate = 2e-5
print("Model is on:", next(model.parameters()).device)
print("Learning rate:", training_args.learning_rate)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
    callbacks=[EarlyStoppingCallback(early_stopping_patience=2)]
)

trainer.train()

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Updated id2label mapping: {0: 'REAL', 1: 'FAKE'}
Updated label2id mapping: {'REAL': 0, 'FAKE': 1}
Model is on: cuda:0
Learning rate: 2e-05


C:\Users\limyi\AppData\Local\Temp\ipykernel_20660\4162383508.py:20: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


Epoch,Training Loss,Validation Loss,Accuracy,F1,Precision,Recall
1,0.596500,0.534173,0.730837,0.709882,0.753711,0.670871
2,0.336700,0.646135,0.730542,0.716501,0.740860,0.693694
3,0.251800,0.713970,0.728184,0.705243,0.753930,0.662462
4,0.247600,0.875297,0.727594,0.689516,0.782609,0.616216


TrainOutput(global_step=6780, training_loss=0.4078136697982968, metrics={'train_runtime': 5504.0966, 'train_samples_per_second': 24.633, 'train_steps_per_second': 1.54, 'total_flos': 2.853807750856704e+16, 'train_loss': 0.4078136697982968, 'epoch': 4.0})

In [6]:
trainer.evaluate()

{'eval_loss': 0.646135151386261,
 'eval_accuracy': 0.7305424528301887,
 'eval_f1': 0.716501240694789,
 'eval_precision': 0.7408595253367544,
 'eval_recall': 0.6936936936936937,
 'eval_runtime': 51.68,
 'eval_samples_per_second': 65.635,
 'eval_steps_per_second': 4.102,
 'epoch': 4.0}

In [12]:
model = AutoModelForSequenceClassification.from_pretrained("./results/checkpoint-2068")
trainer.model = model
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
trainer.model.to(device)

RobertaForSequenceClassification(
  (roberta): RobertaModel(
    (embeddings): RobertaEmbeddings(
      (word_embeddings): Embedding(50265, 768, padding_idx=1)
      (position_embeddings): Embedding(514, 768, padding_idx=1)
      (token_type_embeddings): Embedding(1, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
      (dropout): Dropout(p=0.2, inplace=False)
    )
    (encoder): RobertaEncoder(
      (layer): ModuleList(
        (0-11): 12 x RobertaLayer(
          (attention): RobertaAttention(
            (self): RobertaSdpaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.2, inplace=False)
            )
            (output): RobertaSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
         

In [7]:
# Conduct testing on the test dataset
test_results = trainer.predict(test_dataset)

# Extract predictions and metrics
predictions = test_results.predictions.argmax(-1)  # Convert logits to class predictions
metrics = test_results.metrics  # Contains accuracy, F1, precision, recall, etc.

# Print metrics
print("Test Metrics:")
for key, value in metrics.items():
    print(f"{key}: {value:.4f}")

Test Metrics:
test_loss: 0.6734
test_accuracy: 0.7093
test_f1: 0.6978
test_precision: 0.7266
test_recall: 0.6711
test_runtime: 18.4412
test_samples_per_second: 65.2890
test_steps_per_second: 4.1210


In [8]:
trainer.save_model("./models/oversampled_roberta")

#### With text cleaning (BEFORE TRUE/FALSE SWAP):

| Metric                   | Value     |
|--------------------------|-----------|
| Test Loss                | 0.5277    |
| Test Accuracy            | 0.7287    |
| Test F1 Score            | 0.7106    |
| Test Precision           | 0.6483    |
| Test Recall              | 0.7862    |
| Test Runtime (seconds)   | 34.2732   |
| Test Samples/Second      | 66.9910   |
| Test Steps/Second        | 4.2020    |

#### Without text cleaning (BEFORE TRUE/FALSE SWAP):
| Metric                   | Value     |
|--------------------------|-----------|
| Test Loss                | 0.5339    |
| Test Accuracy            | 0.7395    |
| Test F1 Score            | 0.7051    |
| Test Precision           | 0.6777    |
| Test Recall              | 0.7348    |
| Test Runtime (seconds)   | 39.2421   |
| Test Samples/Second      | 58.5090   |
| Test Steps/Second        | 3.6700    |